In [1]:
from inference_api.visual_attention.manager import VAManager
from inference_api.visual_attention.log_manager import VALogDatabaseManager

/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a

In [2]:
log_manager = VALogDatabaseManager('visual_attention')
va_manager = VAManager(log_manager)

INFO:allennlp.modules.elmo:Initializing ELMo


elmo embedder initialized on gpu?: False


In [3]:
va_manager.infer([0,1])

INFO:root:2022-09-09 17:48:41.323321 - Begin Batch 1


word_image_heatmap
<class 'list'>


indexes
<class 'list'>


image_urls
<class 'list'>


raw_texts
<class 'list'>


text_entity_index
<class 'list'>


image_entity_index
<class 'list'>


token_span
<class 'list'>


bounding_box
<class 'list'>


image_dimensions
<class 'list'>


[(1, 4), (3, 2)]
[(1, 4), (3, 2)]
[(-0.008244834840297699, 1, 1, 1), (-0.007031878922134638, 1, 1, 0)]


{'word_image_heatmap': [tensor([[[[-8.4550e-04,  3.5577e-03, -3.3961e-04, -3.3593e-04],
            [-6.6439e-04,  3.5224e-03, -3.4845e-04, -3.4116e-04],
            [-8.5719e-04,  3.4905e-03, -3.3254e-04, -3.2600e-04],
            ...,
            [-4.3193e-02, -5.3771e-03,  5.0913e-02,  5.4104e-02],
            [-2.7678e-02, -4.3510e-03,  4.5568e-02,  4.2421e-02],
            [-6.3004e-04,  3.8785e-03, -3.4960e-04, -3.4717e-04]],
  
           [[-6.1435e-04,  3.5580e-03, -3.3945e-04, -3.3587e-04],
            [-4.5914e-04,  3.5215e-03, -3.4830e-04, -3.4096e-04],
            [-7.9713e-04,  3.4898e-03, -3.3241e-04, -3.2595e-04],
            ...,
            [-1.5841e-02, -5.3786e-03,  5.0818e-02,  5.4160e-02],
            [-1.5456e-02, -4.3412e-03,  4.5506e-02,  4.2162e-02],
            [-6.3674e-04,  3.8780e-03, -3.4946e-04, -3.4719e-04]],
  
           [[-5.2494e-04,  3.5514e-03, -3.3947e-04, -3.3621e-04],
            [-3.6412e-04,  3.5167e-03, -3.4833e-04, -3.4121e-04],
            

In [ ]:
from pytorch_lightning.utilities.deepspeed import (convert_zero_checkpoint_to_fp32_state_dict)
convert_zero_checkpoint_to_fp32_state_dict('/models/flickr_trained/best.ckpt', '/models/flickr_trained/model.ckpt',tag=None)

In [ ]:
dict_one = {'name': 'John', 'last_name': 'Doe', 'job': 'Python Consultant'}
dict_two = {'name': 'Jane', 'last_name': 'Doe', 'job': 'Community Manager'}
batch = [dict_one.items(),dict_two.items()]

for a in zip(*batch):
    print(a)

In [ ]:
from elasticsearch import Elasticsearch
# from elasticsearch import RequestsHttpConnection

from inference_api.common.inference.data_reader import DataReader

class VALiveDataReader(DataReader):

    ELASTIC_URL = "https://elasticsearch:9200"
#     FIELDS = ['']
    def __init__(self):
        self.client = Elasticsearch(self.ELASTIC_URL,
                                    basic_auth=('elastic', 'changeme'),
                                    verify_certs=False
#                                     connection_class=RequestsHttpConnection
                                    )


    def read(self,document_id):
        result = self.client.get(index='documents',
                                 id=document_id,
#                                  stored_fields =self.FIELDS
                                )
        return result

reader = VALiveDataReader()
result = reader.read("141dc5e3e4e399c880f5d96f03c07c0d")
text = result['_source']['content']
text_content = result['_source']['content']
visual_entities = result['_source']['visual_entities']
text_entities = result['_source']['text_entities']

In [ ]:
visual_entities

In [ ]:
entity

In [ ]:
bounding_boxes = visual_entities[0]['person_bbox']
bounding_boxes

In [ ]:

import json
visual_entities = result['_source']['visual_entities']

# visual_entities

image_url = visual_entities[0]['file_name']

entity_ids = visual_entities[0]['person_id']
bounding_boxes = visual_entities[0]['person_bbox']


import requests
import numpy as np
server_path = image_url
body = {'server_path': server_path}
r = requests.get(
        'http://image_server:8000/download/', json=body)

image = np.asarray(r.json()['image'])


print(image.shape)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
fig, ax = plt.subplots()

# Display the image
ax.imshow(image)

# Create a Rectangle patch
bounding_box = bounding_boxes[0]
top_right_x = max(bounding_box[0],0)
top_right_y = max(bounding_box[1],0)

delta_x = max(bounding_box[2],0) -top_right_x 
delta_y = max(bounding_box[3],0) - top_right_y


rect = patches.Rectangle((top_right_x, top_right_y), delta_x, delta_y, linewidth=1, edgecolor='r', facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

plt.show()


In [ ]:


import json

import requests
import numpy as np


# class VisualTextEntityExtractor:
from elasticsearch import Elasticsearch


from inference_api.common.inference.data_reader import DataReader
class VALiveDataReader(DataReader):

    ELASTIC_URL = "https://elasticsearch:9200"

    def __init__(self):
        self.text_entity_extractor = UnknownTextEntityExtractor()
        self.visual_entity_extractor = UnknownVisualEntityExtractor()
        self.client = Elasticsearch(self.ELASTIC_URL,
                                    basic_auth=('elastic', 'changeme'),
                                    verify_certs=False
                                    )

    def get_generator(self, document_ids):
        index = 0
        for document_id in document_ids:
            result = self.client.get(index='documents',
                                     id=document_id,
                                     )

            text_content = result['_source']['content']
            visual_entities = result['_source']['visual_entities']
            text_entities = result['_source']['text_entities']

            text_generator = self.text_entity_extractor.get_generator(
                text_content, text_entities)
            image_generator = self.visual_entity_extractor.get_generator(
                visual_entities)

            for image_url, image_data, bounding_box in image_generator:
                for text, token_span in text_generator:
                    yield {
                        'index': index,
                        'image_url': image_url,
                        'text': text,
                        'image': image_data,
                        'token_span': token_span,
                        'bounding_box': bounding_box}


        
            
        
class UnknownVisualEntityExtractor:
    
    
    def __init__(self):
         pass
    def get_generator(self,images): 
        
        
        for image in images:
            entity_ids = image['person_id']
            bounding_boxes = image['person_bbox']
            image_url = image['file_name']
            
            image_data = self.download_image(image_url)
            N = len(entity_ids)
            for i in range(N):
                entity_id = entity_ids[i]
                if entity_id == -1:
                    yield image_url,image_data,bounding_boxes[i]
    
    def download_image(self,server_path):
        body = {'server_path': server_path}
        r = requests.get('http://image_server:8000/download/', json=body)
        image = np.asarray(r.json()['image'])
        return image
 
class UnknownTextEntityExtractor:
    
    def __init__(self):
        self.token_mapper = TextTokenMapper()
        
    def get_generator(self,text,text_entities):
        N = len(text_entities)
        sentences = self.token_mapper.split_sentences(text)
        for i in range(N):
            entity = text_entities[i]
            if entity['entity_link'] == -1:
                sentence_index,span_start,span_end = entity['mention_span']
                sentence = sentences[sentence_index]
                token_span = self.token_mapper.get_tokens(sentence,entity['mention'],span_start,span_end)
                yield sentences[sentence_index],token_span
                

from spacy.lang.en import English

class TextTokenMapper:
    
    def __init__(self):
        self.sentencizer = English()
        self.sentencizer.add_pipe('sentencizer')
    
    def split_sentences(self,text):
        paras = text.split("\n")
        sentences = []
        for para in paras:
            str_sents = list(self.sentencizer(para).sents)
            for sent in str_sents:
                tokens = list(self.sentencizer.tokenizer(sent.text))
                tokens = [token.text for token in tokens]
                if len(tokens) > 0:
                    sentences.append(sent.text)
        return sentences
    
    def get_tokens(self,sentence,entity_mention,span_start,span_end):
        tokens = sentence.split(' ')
        N = len(tokens)
        entity_tokens = len(entity_mention.split(' '))
        cumilative_char_index = 0
        char_to_token_span = []

        for token_index, token in enumerate(tokens):
            end_char_index = cumilative_char_index + len(token)
            char_to_token_span.append((cumilative_char_index,end_char_index,token_index))
            cumilative_char_index = end_char_index + 1                
        left = 0
        char_2_token_spans = {}
        for right in range(entity_tokens-1,N):
            left_start_index, left_end_index, left_token_index = char_to_token_span[left]
            right_start_index,right_end_index, right_token_index = char_to_token_span[right]
            char_2_token_spans[(left_start_index,right_end_index)] = (left_token_index,right_token_index) 
            left += 1


        while span_start > 0 and sentence[span_start - 1] != ' ':
            span_start -= 1
        while span_end < len(sentence) and sentence[span_end] != ' ':
            span_end += 1
       
        
        token_span = char_2_token_spans[(span_start,span_end)]
        

        return token_span
    
    
# text_ext = UnknownTextEntityExtractor()
# for i in text_ext.get_generator(text,text_entities):
#     print(i[0])



reader = VALiveDataReader()
generator = reader.get_generator(["141dc5e3e4e399c880f5d96f03c07c0d"])


for data in generator:
    print(data)
    break
        

In [ ]:
'/.a'.isalnum()

In [ ]:
visual_entities

In [ ]:
text_entities = result['_source']['text_entities']
i= 13
entity = text_entities[i]
# span = entity['mention_span']
# mention_text = entity['mention']
text = result['_source']['content']
sentence_index,span_start,span_end = entity['sentence_char_span']



text = result['_source']['content']


In [ ]:

    
token_mapper = TextTokenMapper()

sentences = token_mapper.split_sentences(text)

token_mapper.get_tokens(sentences,entity['mention'],entity['sentence_char_span'])



In [ ]:
text_entities = result['_source']['text_entities']
text_entities

In [ ]:
import re
content = result['_source']['content']
re.split(' |\n', content)

# span

In [ ]:
result['_source']['content'].split(' ')

In [ ]:
text_entities

In [ ]:
result['_source']['content']